In [3]:
# STANDARD LIBRARY IMPORTS
import re
import csv
import os
import datetime

# PROJECT IMPORTS
from setup import reddit
from settings import *

# THIRD PARTY IMPORTS
import praw

Loaded 2 users who have opted out.


In [4]:
'Did you get good feedback on your photo in /r/{TARGET_SUBREDDIT}?'

'Did you get good feedback on your photo in /r/{TARGET_SUBREDDIT}?'

In [5]:
'''
You recently posted [a photo for critique](https://old.reddit.com/r/phcritiquetest/comments/szykjh/i_took_this_photo_of_my_tent_in_the_snow_a_few/) in /r/photocritique.

I hope you received some useful feedback -- if you did, you can reply to any comment with '!goodcritique' to give a Critique Point to the person who provided you with a high quality critique. Giving Critique Points is easy, and helps improve the whole community.
See [here](todo) for more information on Critique Points.

This message was sent by a bot. To opt out of future reminders such as this one, reply to this PM with 'optout'.
'''

"\nYou recently posted [a photo for critique](https://old.reddit.com/r/phcritiquetest/comments/szykjh/i_took_this_photo_of_my_tent_in_the_snow_a_few/) in /r/photocritique.\n\nI hope you received some useful feedback -- if you did, you can reply to any comment with '!goodcritique' to give a Critique Point to the person who provided you with a high quality critique. Giving Critique Points is easy, and helps improve the whole community.\nSee [here](todo) for more information on Critique Points.\n\nThis message was sent by a bot. To opt out of future reminders such as this one, reply to this PM with 'optout'.\n"

In [6]:
subreddit = reddit.subreddit(TARGET_SUBREDDIT)

In [7]:
# overiding this for testing
REMINDER_LOWER_BOUND, REMINDER_UPPER_BOUND = 24,4*24+6
assert REMINDER_LOWER_BOUND <= REMINDER_UPPER_BOUND

In [9]:
now = datetime.datetime.utcnow() # using utcnow to align with reddit timezone

for submission in subreddit.new(limit=None):
    submission_age = now - datetime.datetime.fromtimestamp( submission.created_utc )
    submission_age = submission_age.total_seconds() / 60**2 # convert to total hours
    
    if submission_age < REMINDER_LOWER_BOUND:
        print(f'skipping {submission.title}')
        continue # ignore young posts
    
    # process posts here
    print(f'processing {submission.title}')
    
    if submission_age > REMINDER_UPPER_BOUND:
        print(f'stopping at older post {submission.title}')
        break

skipping Grain elevators in the Palouse in winter.
processing Mount Rainier from Mount Pleasant
stopping at older post Mount Rainier from Mount Pleasant


In [14]:
datetime.datetime.fromtimestamp( submission.created_utc )

datetime.datetime(2022, 2, 25, 18, 24, 35)

In [19]:
submission_age.total_seconds() / 60**2

126.2050830475

In [11]:
submission.flair.choices()

[{'flair_css_class': '',
  'flair_template_id': '7eeec638-8f07-11ec-8294-2a47d40d3100',
  'flair_text_editable': True,
  'flair_position': 'right',
  'flair_text': 'HPs from op'}]